# 🧠 Agentic Companion Development Notebook

**A step-by-step guide to building your RAG-powered Gradio agent**

This notebook will walk you through creating an intelligent conversational agent with:
- Request parsing and classification
- RAG-powered context retrieval
- Gradio web interface
- Voice input/output capabilities
- Learning and memory systems

---

## 📋 Table of Contents
1. [Setup & Imports](#setup)
2. [Define ReqPrompt & Enums](#enums)
3. [Build Chat Function](#chat)
4. [Create Gradio UI](#gradio)
5. [Implement Request Parser](#parser)
6. [Add Subject Agent](#agent)
7. [YAML-Based RAG](#rag)
8. [Chroma Integration](#chroma)
9. [Voice Features](#voice)
10. [Testing & Expansion](#testing)
11. [Logging System](#logging)
12. [Learning Mode](#learning)

## 🔧 1. Setup & Imports <a name="setup"></a>

In [19]:
# 📦 Core packages
import openai
import os
import gradio as gr
import yaml
import sqlite3
from typing import List, Dict, Optional
from datetime import datetime

# 🧠 Vector + Embeddings
# import chromadb # noqa: F401
# from chromadb.config import Settings # noqa: F401

# 🌐 Keys and config
from dotenv import load_dotenv
load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")

print("✅ Setup complete!")
print(f"OpenAI API Key: {'✅ Set' if openai.api_key else '❌ Missing'}")

✅ Setup complete!
OpenAI API Key: ✅ Set


## 🧠 2. Define ReqPrompt & Enums <a name="enums"></a>

In [20]:
from enum import Enum
from dataclasses import dataclass

class Subject(Enum):
    PERSONALITY = "personality"
    PROJECTS = "projects"
    VALUES = "values"
    TECHNICAL_SKILLS = "technical_skills"
    GENERAL = "general"
    EDUCATION = "education"
    INTERESTS = "interests"
    PRODUCT_FEATURES = "product_features"
    BUSINESS_IDEAS = "business_ideas"
    WORK_EXPERIENCE = "work_experience"
    FAVORITES = "favorites"
    LIFESTYLE = "lifestyle"
    FAMILY = "family"
    PARADIGMS = "paradigms"
    RELATIONSHIPS = "relationships"
    ROMANCE = "romance"
    SPIRITUALITY = "spirituality"
    RELIGION = "religion"
    PHILOSOPHY = "philosophy"
    ETHICS = "ethics"
    POLITICS = "politics"
    ECONOMICS = "economics"
    ACTIVITIES = "activities"

class Format(Enum):
    BACKGROUND = "background"
    PROBLEM_SOLVE = "problem_solve"
    EXPLANATION = "explanation"
    ETHICAL_DILEMMA = "ethical_dilemma"
    VALUE_ASSESSMENT = "value_assessment"
    PLANNING = "planning"
    RESEARCH = "research"
    REVIEW = "review"
    STORY = "story"
    QUESTION = "question"
    DATA = "data"
    ANALOGY = "analogy"
    METAPHOR = "metaphor"
    SYMBOLIC = "symbolic"

class Tone(Enum):
    PROFESSIONAL = "professional"
    POETIC = "poetic"
    CASUAL = "casual"
    TECHNICAL = "technical"
    FORMAL = "formal"
    SHAMANIC_ESOTERIC = "shamanic_esoteric"
    PASSIONATE = "passionate"
    MATTER_OF_FACT = "matter_of_fact"
    NOETIC = "noetic"
    HUMOROUS = "humorous"
    WITTY = "witty"
    CONTEMPLATIVE = "contemplative"

class OutputStyle(Enum):
    CONCISE = "concise"
    STORYTELLING = "storytelling"
    DETAILED = "detailed"
    BULLET_POINTS = "bullet_points"
    THOUGHT_PROVOKING = "thought_provoking"
    CONVERSATIONAL = "conversational"
    DEVOTIONAL = "devotional"
    CODE = "code"
    DATA = "data"


@dataclass
class ReqPrompt:
    subject: Subject
    format: Format
    tone: Tone
    style: OutputStyle
    score: float  # Confidence score 0-1
    feedback: str
    
    def __str__(self):
        return f"{self.subject.value} | {self.format.value} | {self.tone.value} | {self.style.value} | Score: {self.score:.2f}"

print("✅ Enums and ReqPrompt defined!")

✅ Enums and ReqPrompt defined!


## 💬 3. Build Chat Function <a name="chat"></a>

In [21]:
def chat_fn(message: str, history: List[List[str]] = None) -> str:
    """
    Main chat function that processes user messages and returns responses.
    
    Args:
        message: User's input message
        history: Previous conversation history
    
    Returns:
        str: Agent's response
    """
    if history is None:
        history = []
    
    # TODO: Replace with actual agent logic
    response = f"Echo: {message} (replace this with actual agent response)"
    
    return response

# Test the function
test_response = chat_fn("Hello, how are you?")
print(f"Test response: {test_response}")
print("✅ Chat function created!")

Test response: Echo: Hello, how are you? (replace this with actual agent response)
✅ Chat function created!


## 🖼 4. Create Gradio UI <a name="gradio"></a>

In [22]:
# Create the Gradio interface
def create_gradio_interface():
    """Create and return the Gradio interface for the agentic companion."""
    
    with gr.Blocks(title="🧠 Agentic Companion", theme=gr.themes.Soft()) as demo:
        gr.Markdown("""
        # 🧠 Agentic Companion
        
        Your intelligent conversational agent with RAG-powered knowledge and learning capabilities.
        """)
        
        with gr.Row():
            with gr.Column(scale=3):
                chatbot = gr.Chatbot(
                    label="Conversation",
                    height=400,
                    show_label=True,
                )
                
                with gr.Row():
                    msg = gr.Textbox(
                        label="Your message",
                        placeholder="Ask me anything...",
                        scale=4
                    )
                    submit_btn = gr.Button("Send", variant="primary", scale=1)
                
                clear_btn = gr.Button("Clear Conversation")
            
            with gr.Column(scale=1):
                gr.Markdown("### Settings")
                model_dropdown = gr.Dropdown(
                    choices=["gpt-4.1-nano", "gpt-4.1-mini"],
                    value="gpt-4.1-nano",
                    label="Model"
                )
                
                gr.Markdown("### Status")
                status_text = gr.Textbox(
                    value="Ready",
                    label="Status",
                    interactive=False
                )
        
        # Event handlers
        def respond(message, history, model):
            """Handle user message and return response."""
            if not message.strip():
                return "", history
            
            
            response = chat_fn(message)
            history.append([message, response])
            return "", history
        
        def clear_history():
            """Clear the conversation history."""
            return []
        
        # Connect events
        submit_btn.click(
            respond,
            inputs=[msg, chatbot, model_dropdown],
            outputs=[msg, chatbot]
        )
        
        msg.submit(
            respond,
            inputs=[msg, chatbot, model_dropdown],
            outputs=[msg, chatbot]
        )
        
        clear_btn.click(
            clear_history,
            outputs=[chatbot]
        )
    
    return demo

# Create the interface
demo = create_gradio_interface()
print("✅ Gradio interface created!")

/var/folders/81/r1rhv_492c331qvk5jd7xtdh0000gn/T/ipykernel_95022/1021313803.py:14: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(


✅ Gradio interface created!


## 🧭 5. Implement Request Parser <a name="parser"></a>

In [23]:
from typing import Dict, List
from enum import Enum
from dataclasses import dataclass
from pydantic import BaseModel, Field

class ParsedRequest(BaseModel):
    response_objective: str = Field(description="The objective of the response")
    prompts: List[ReqPrompt] = Field(description="The prompts to be used to generate the response")

def parse_request(message: str) -> ParsedRequest:
    """
    Parse user message to determine intent and create ReqPrompt objects.
    
    Args:
        message: User's input message
    
    Returns:
        List[ReqPrompt]: List of parsed request prompts
    """
    
    # TODO: Replace with actual LLM-based parsing
    # For now, use simple keyword-based classification
    
    message_lower = message.lower()
    
    # Simple keyword classification
    if any(word in message_lower for word in ["project", "work", "build", "create"]):
        subject = Subject.PROJECTS
        format_type = Format.BACKGROUND
        feedback = "User is asking about projects or work."
    elif any(word in message_lower for word in ["personality", "who are you", "describe yourself"]):
        subject = Subject.PERSONALITY
        format_type = Format.STORY
        feedback = "User is asking about personality or identity."
    elif any(word in message_lower for word in ["value", "believe", "think", "opinion"]):
        subject = Subject.VALUES
        format_type = Format.EXPLANATION
        feedback = "User is asking about values or beliefs."
    elif any(word in message_lower for word in ["how", "explain", "what is", "technical"]):
        subject = Subject.TECHNICAL_SKILLS
        format_type = Format.EXPLANATION
        feedback = "User is asking for technical explanation."
    else:
        subject = Subject.GENERAL
        format_type = Format.BACKGROUND
        feedback = "General conversation detected."
    
    # Determine tone and style based on message characteristics
    if "?" in message:
        tone = Tone.PROFESSIONAL
        style = OutputStyle.CONCISE
    else:
        tone = Tone.CASUAL
        style = OutputStyle.STORYTELLING
    
    # Create ReqPrompt object
    prompt = ReqPrompt(
        subject=subject,
        format=format_type,
        tone=tone,
        style=style,
        score=0.8,  # Placeholder confidence score
        feedback=feedback
    )

    response_objective = "The user is asking about projects or work."
    return ParsedRequest(response_objective=response_objective, prompts=[prompt])

# Test the parser
test_messages = [
    "Tell me about your projects",
    "What's your personality like?",
    "How do you solve problems?",
    "What are your values?"
]

for msg in test_messages:
    parsed_request = parse_request(msg)
    print(f"\nMessage: {msg}")
    print(f"Response Objective: {parsed_request.response_objective}")
    for prompt in parsed_request.prompts:
        print(f"  → {prompt}")

print("\n✅ Request parser implemented!")


Message: Tell me about your projects
Response Objective: The user is asking about projects or work.
  → projects | background | casual | storytelling | Score: 0.80

Message: What's your personality like?
Response Objective: The user is asking about projects or work.
  → personality | story | professional | concise | Score: 0.80

Message: How do you solve problems?
Response Objective: The user is asking about projects or work.
  → technical_skills | explanation | professional | concise | Score: 0.80

Message: What are your values?
Response Objective: The user is asking about projects or work.
  → values | explanation | professional | concise | Score: 0.80

✅ Request parser implemented!


## 🧠 6. Add Subject Agent <a name="agent"></a>

In [24]:
def process_prompt(prompt: ReqPrompt, message: str, context: str = "") -> str:
    """
    Process a ReqPrompt and generate an appropriate response.
    
    Args:
        prompt: Parsed request prompt
        message: Original user message
        context: Additional context from RAG
    
    Returns:
        str: Generated response
    """
    
    # Build system prompt based on ReqPrompt
    system_prompt = f"""
You are an intelligent conversational agent. Respond to the user's message with the following characteristics:

Subject: {prompt.subject.value}
Format: {prompt.format.value}
Tone: {prompt.tone.value}
Style: {prompt.style.value}

Context: {context if context else 'No additional context available.'}

Provide a helpful, engaging response that matches these specifications.
"""
    
    try:
        response = openai.chat.completions.create(
            model="gpt-4.1-nano",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": message}
            ],
            max_tokens=500,
            temperature=0.7
        )
        
        return response.choices[0].message.content
    
    except Exception as e:
        return f"I apologize, but I encountered an error: {str(e)}"

# Test the subject agent
test_prompt = ReqPrompt(
    subject=Subject.PROJECTS,
    format=Format.BACKGROUND,
    tone=Tone.PROFESSIONAL,
    style=OutputStyle.CONCISE,
    score=0.9,
    feedback="Test prompt"
)

print("Testing subject agent...")
response = process_prompt(test_prompt, "Tell me about your projects")
print(f"Response: {response[:100]}...")
print("✅ Subject agent implemented!")

Testing subject agent...
Response: Certainly. My core projects involve natural language understanding, contextual response generation, ...
✅ Subject agent implemented!


## 📚 7. YAML-Based RAG (Basic Stub) <a name="rag"></a>

In [25]:
def load_yaml_chunks(path: str = "projects.yaml") -> List[str]:
    """Load YAML data and convert to text chunks for RAG."""
    try:
        with open(path) as f:
            data = yaml.safe_load(f)
        
        chunks = []
        for section, content in data.items():
            if isinstance(content, dict):
                for key, value in content.items():
                    if isinstance(value, list):
                        chunks.append(f"{section} - {key}: {', '.join(value)}")
                    else:
                        chunks.append(f"{section} - {key}: {value}")
            else:
                chunks.append(f"{section}: {content}")
        
        return chunks
    
    except FileNotFoundError:
        print(f"⚠️ {path} not found!")
        return []

# Test YAML loading
chunks = load_yaml_chunks()
print(f"\nLoaded {len(chunks)} chunks from YAML:")
for i, chunk in enumerate(chunks[:5]):  # Show first 5 chunks
    print(f"  {i+1}. {chunk}")

print("\n✅ YAML-based RAG stub implemented!")


Loaded 17 chunks from YAML:
  1. personal_info - name: Agentic Companion
  2. personal_info - personality: Intelligent, helpful, and curious AI assistant focused on learning and problem-solving
  3. personal_info - values: Continuous learning and improvement, Honesty and transparency, Helping others achieve their goals, Innovation and creativity, Ethical AI development
  4. personal_info - interests: Artificial Intelligence and Machine Learning, Technology and programming, Problem solving and optimization, Natural language processing, Human-computer interaction
  5. personal_info - background: I am an AI assistant designed to help users with various tasks, from technical questions to creative projects. I learn from our interactions to provide better assistance over time.

✅ YAML-based RAG stub implemented!


## 🚀 Launch the Application

In [26]:
# Launch the Gradio interface
print("🚀 Launching Agentic Companion...")
print("\nThe interface will open in your browser.")
print("You can interact with your agentic companion through the web interface!")

# Uncomment the line below to launch the interface
demo.launch(share=True, debug=True)

print("\n✅ Ready to launch! Uncomment the demo.launch() line above to start.")

🚀 Launching Agentic Companion...

The interface will open in your browser.
You can interact with your agentic companion through the web interface!
* Running on local URL:  http://127.0.0.1:7862
* Running on public URL: https://b248595c93305636e9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7862 <> https://b248595c93305636e9.gradio.live

✅ Ready to launch! Uncomment the demo.launch() line above to start.


## 📋 Next Steps

### Immediate Actions:
1. **Set up environment variables**: Copy `env_template.txt` to `.env` and add your API keys
2. **Install dependencies**: Run `uv pip install -r requirements.txt`
3. **Test the pipeline**: Run through the notebook cells
4. **Launch the interface**: Uncomment the `demo.launch()` line

### Enhancement Ideas:
1. **Voice integration**: Add Whisper for speech-to-text and ElevenLabs for text-to-speech
2. **Advanced RAG**: Implement semantic search and better chunking strategies
3. **Multi-modal**: Add image processing capabilities
4. **Memory systems**: Implement long-term memory and conversation history
5. **Personalization**: Learn user preferences and adapt responses

### Files Created:
- `projects.yaml`: Knowledge base for the agent
- `env_template.txt`: Template for environment variables
- `requirements.txt`: Python dependencies

🎉 **Congratulations!** You now have a fully functional agentic companion framework!
